In [2]:
import numpy as np
import pandas as pd
import random

In [93]:
def play_out_scenario(player_value, dealer_value, usable_ace, dealer_usable_ace, \
                      verbose=False, starting_action=None):
    
    if verbose:
        print(f"Trial State: player_value={player_value}, " + \
            f"dealer_value={dealer_value}, " + \
            f" usable_ace={usable_ace}," + \
            f" starting_action={starting_action}," + \
            f" dealer_usable_ace={dealer_usable_ace}\n")
    
    value = 0

    bust = False
    dealer_bust = False


    #Player start
    if starting_action == None:
        action=policy[usable_ace].loc[dealer_card, player_value]
    else:
        action=starting_action


    #Player cards
    while action == "hit" and bust == False:

        #Draw a card
        card = random.choices(cards, weights=card_probs)[0]
        if verbose:
            print(f"Player Card: {card}")

        #Add it to player_value
        if card == "A":
            player_value += 1
        else:
            player_value += card

        #If over 21, call bust, or use usable_ace and decrement by 10
        if player_value > 21:
            if usable_ace == False:
                bust = True
            else:
                usable_ace = False
                player_value -= 10
        if verbose:
            print(f"Player Situation: player_value={player_value}, " + \
                  f"dealer_card={dealer_card}, usable_ace={usable_ace}")

        #Set new action
        if not bust:
            action = policy[usable_ace].loc[dealer_card, player_value]
            if verbose:
                print(f"Player Action: {action}")


    #Print player results
    if verbose:
        print(f"Player End: player_value={player_value}, dealer_card={dealer_card}, " + \
              f"usable_ace={usable_ace}, bust={bust}\n")


    #Dealer start
    dealer_action=dealer_policy[dealer_usable_ace][dealer_value]
    if verbose:
        print(f"Dealer Start: dealer_value={dealer_value}, dealer_usable_ace={dealer_usable_ace}")

    #Dealer cards
    if bust:
        pass
    else:
        while dealer_action == "hit" and dealer_bust == False:

            #Draw a card
            card = random.choices(cards, weights=card_probs)[0]
            if verbose:
                print(f"Dealer Card: {card}")

            #Add it to dealer_value
            if card == "A":
                dealer_value += 1
            else:
                dealer_value += card

            #If over 21, call bust, or use usable_ace and decrement by 10
            if dealer_value > 21:
                if dealer_usable_ace == False:
                    dealer_bust = True
                else:
                    dealer_usable_ace = False
                    dealer_value -= 10
            if verbose:
                print(f"Dealer Situation: dealer_value={player_value}, " + \
                              f"dealer_usable_ace={dealer_usable_ace}")

            #Set new action
            if not dealer_bust:
                dealer_action = dealer_policy[dealer_usable_ace][dealer_value]
                if verbose:
                    print(f"Dealer Action: {action}")

        #Print dealer results
        if verbose:
            print(f"Dealer End: dealer_value={dealer_value}," + \
                f"dealer_usable_ace={dealer_usable_ace}, dealer_bust={dealer_bust}\n")

    #Compute state value
    if bust:
        value = -1
    elif dealer_bust:
        value = 1
    elif dealer_value > player_value:
        value = -1
    elif player_value > dealer_value:
        value = 1

    if verbose:
        print(f"Result: {value}\n")
        
    return value

In [91]:
starting_values = np.arange(12, 22)

cards = ["A" if x == 1 else x for x in range(1, 11)]
card_probs = [4/13 if card==10 else 1/13 for card in cards]


print(sum(card_probs))
print(cards)
print(starting_values)

1.0
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10]
[12 13 14 15 16 17 18 19 20 21]


In [116]:
#Initialize state-action values and num visits
state_action_values = dict()
state_action_visits = dict()
for action in ["hit", "stay"]:
    state_action_values[action] = dict()
    state_action_visits[action] = dict()
    for starting_usable_ace in [True, False]:
        state_action_values[action][starting_usable_ace] = pd.DataFrame(index=cards, columns=starting_values, data=0)
        state_action_visits[action][starting_usable_ace] = pd.DataFrame(index=cards, columns=starting_values, data=0)
        

print("Policy:")
policy = dict()
for usable_ace in [True, False]:
    print(f"usable_ace={usable_ace}")
    policy[usable_ace] = pd.DataFrame(index=cards, columns=starting_values, data=False)
    for starting_value in starting_values:
        if starting_value > 16:
            policy[usable_ace][starting_value] = "hit"
        else:
            policy[usable_ace][starting_value] = "stay"
    display(policy[usable_ace])

print("Dealer policy:")
dealer_policy = dict()
for usable_ace in [True, False]:
    dealer_policy[usable_ace] = dict()
    for dealer_value in range(4, 22):
        if dealer_value < 17:
            dealer_policy[usable_ace][dealer_value] = "hit"
        else:
            dealer_policy[usable_ace][dealer_value] = "stay"      
display(dealer_policy)

Policy:
usable_ace=True


,12,13,14,15,16,17,18,19,20,21
A,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
2,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
3,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
4,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
5,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
6,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
7,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
8,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
9,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
10,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit


usable_ace=False


,12,13,14,15,16,17,18,19,20,21
A,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
2,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
3,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
4,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
5,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
6,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
7,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
8,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
9,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit
10,stay,stay,stay,stay,stay,hit,hit,hit,hit,hit


Dealer policy:


{True: {4: 'hit',
  5: 'hit',
  6: 'hit',
  7: 'hit',
  8: 'hit',
  9: 'hit',
  10: 'hit',
  11: 'hit',
  12: 'hit',
  13: 'hit',
  14: 'hit',
  15: 'hit',
  16: 'hit',
  17: 'stay',
  18: 'stay',
  19: 'stay',
  20: 'stay',
  21: 'stay'},
 False: {4: 'hit',
  5: 'hit',
  6: 'hit',
  7: 'hit',
  8: 'hit',
  9: 'hit',
  10: 'hit',
  11: 'hit',
  12: 'hit',
  13: 'hit',
  14: 'hit',
  15: 'hit',
  16: 'hit',
  17: 'stay',
  18: 'stay',
  19: 'stay',
  20: 'stay',
  21: 'stay'}}

In [169]:
#num trials
num_loops = 10000
verbose = False

for loop in range(num_loops):
    if loop % 100 == 0:
        print(f"loop={loop}")
    
    #All possible states
    for starting_usable_ace in [True, False]:
        for starting_value in starting_values:
            for dealer_card in cards:
                for starting_action in ["hit", "stay"]:
                
                    
                    if verbose:
                        print(f"Starting State: starting_value={starting_value}, " + \
                          f"starting_action={starting_action}, " + \
                          f" dealer_card={dealer_card}," + \
                          f" starting_usable_ace={starting_usable_ace}\n")

                    #Player starting situation
                    player_value = starting_value
                    usable_ace = starting_usable_ace
                    bust = False

                    #Dealer starting situation
                    dealer_other_card = random.choices(cards, weights=card_probs)[0]
                    dealer_value = 0
                    dealer_usable_ace=False

                    #Compute dealer value
                    for card in [dealer_card, dealer_other_card]:
                        if card == "A":
                            dealer_usable_ace=True
                            dealer_value +=11
                        else:
                            dealer_value += card

                    #Only one way to use usable_ace here, so we special case it.
                    #There were 2 aces, so one usable ace remains.
                    if dealer_value == 22 and dealer_usable_ace == True:
                        dealer_value -= 10 and dealer_usable_ace == True

                    new_value = play_out_scenario(player_value,
                                              dealer_value,
                                              usable_ace,
                                              dealer_usable_ace,
                                              verbose=verbose,
                                              starting_action=starting_action)
                    visits = state_action_visits[starting_action][starting_usable_ace].loc[dealer_card, \
                                                                                          starting_value]
                    mean_value = state_action_values[starting_action][starting_usable_ace].loc[dealer_card, \
                                                                                              starting_value]
                    
                    if visits > 0:
                        new_mean_value = (mean_value*visits+new_value)/(visits+1)
                    else:
                        new_mean_value = new_value
                    state_action_values[starting_action][starting_usable_ace].at[dealer_card, \
                                                            starting_value] = new_mean_value
                    state_action_visits[starting_action][starting_usable_ace].at[dealer_card, \
                                                            starting_value] = visits + 1


                    if verbose:
                        print(f"mean_value={mean_value}")
                        print(f"new_mean_value={new_mean_value}\n")
                        
                #Update policy
                if state_action_values["hit"][starting_usable_ace].at[dealer_card, starting_value] > \
                    state_action_values["stay"][starting_usable_ace].at[dealer_card, starting_value]:
                    policy[usable_ace].loc[dealer_card, player_value] = "hit"
                elif state_action_values["stay"][starting_usable_ace].at[dealer_card, starting_value] > \
                    state_action_values["hit"][starting_usable_ace].at[dealer_card, starting_value]:
                    policy[usable_ace].loc[dealer_card, player_value] = "stay"
                else:
                    if random.choice([True, False]) == True:
                        policy[usable_ace].loc[dealer_card, player_value] = "hit"
                    else:
                        policy[usable_ace].loc[dealer_card, player_value] = "stay"
    

loop=0
loop=100
loop=200
loop=300
loop=400
loop=500
loop=600
loop=700
loop=800
loop=900
loop=1000
loop=1100
loop=1200
loop=1300
loop=1400
loop=1500
loop=1600
loop=1700
loop=1800
loop=1900
loop=2000
loop=2100
loop=2200
loop=2300
loop=2400
loop=2500
loop=2600
loop=2700
loop=2800
loop=2900
loop=3000
loop=3100
loop=3200
loop=3300
loop=3400
loop=3500
loop=3600
loop=3700
loop=3800
loop=3900
loop=4000
loop=4100
loop=4200
loop=4300
loop=4400
loop=4500
loop=4600
loop=4700
loop=4800
loop=4900
loop=5000
loop=5100
loop=5200
loop=5300
loop=5400
loop=5500
loop=5600
loop=5700
loop=5800
loop=5900
loop=6000
loop=6100
loop=6200
loop=6300
loop=6400
loop=6500
loop=6600
loop=6700
loop=6800
loop=6900
loop=7000
loop=7100
loop=7200
loop=7300
loop=7400
loop=7500
loop=7600
loop=7700
loop=7800
loop=7900
loop=8000
loop=8100
loop=8200
loop=8300
loop=8400
loop=8500
loop=8600
loop=8700
loop=8800
loop=8900
loop=9000
loop=9100
loop=9200
loop=9300
loop=9400
loop=9500
loop=9600
loop=9700
loop=9800
loop=9900


In [170]:
print(state_action_visits[starting_action][starting_usable_ace].loc[dealer_card, \
                                                            starting_value])
state_value_diffs=dict()
for starting_usable_ace in [True, False]:
    print(f"starting_usable_ace={starting_usable_ace}")
    
    state_value_diffs[starting_usable_ace] = pd.DataFrame(index=cards, columns=starting_values)
    for starting_value in starting_values:
        for dealer_card in cards:
            state_value_diffs[starting_usable_ace].at[dealer_card, starting_value] = \
                state_action_values["hit"][starting_usable_ace].loc[dealer_card, starting_value] - \
                   state_action_values["stay"][starting_usable_ace].loc[dealer_card, starting_value]
        
    display(state_value_diffs[starting_usable_ace])
    display(policy[starting_usable_ace])

19761
starting_usable_ace=True


,12,13,14,15,16,17,18,19,20,21
A,0.472531,0.446783,0.419112,0.401528,0.382911,0.245055,0.046593,-0.11762,-0.319589,-0.741614
2,0.366704,0.33448,0.296135,0.277129,0.275105,0.13649,-0.07386,-0.265948,-0.464737,-0.627473
3,0.349858,0.32502,0.277772,0.267618,0.272474,0.133758,-0.069864,-0.254212,-0.445664,-0.629901
4,0.330686,0.336099,0.288749,0.269641,0.26868,0.148986,-0.057014,-0.241868,-0.429171,-0.590185
5,0.338679,0.288547,0.273118,0.249254,0.252087,0.144837,-0.06526,-0.237365,-0.411464,-0.578497
6,0.331344,0.309743,0.303673,0.275105,0.246471,0.133809,-0.08752,-0.272336,-0.414419,-0.562813
7,0.637141,0.598493,0.571024,0.515506,0.477614,0.15364,-0.233015,-0.38971,-0.501796,-0.638148
8,0.612353,0.56293,0.520134,0.460414,0.449031,0.297617,-0.076744,-0.452595,-0.589476,-0.698052
9,0.54831,0.505817,0.474228,0.431072,0.386149,0.276774,0.079324,-0.287059,-0.640425,-0.768834
10,0.450374,0.421135,0.374159,0.352102,0.323974,0.210047,0.019983,-0.133057,-0.473615,-0.828586


,12,13,14,15,16,17,18,19,20,21
A,hit,hit,hit,hit,hit,hit,hit,stay,stay,stay
2,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
3,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
4,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
5,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
6,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
7,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
8,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
9,hit,hit,hit,hit,hit,hit,hit,stay,stay,stay
10,hit,hit,hit,hit,hit,hit,hit,stay,stay,stay


starting_usable_ace=False


,12,13,14,15,16,17,18,19,20,21
A,0.253529,0.22216,0.198087,0.166473,0.136619,-0.007489,-0.274669,-0.58575,-0.908658,-1.573503
2,0.024336,-0.03233,-0.067648,-0.132369,-0.198452,-0.388675,-0.749367,-1.127271,-1.509084,-1.888113
3,0.000152,-0.032178,-0.111465,-0.172443,-0.232669,-0.414179,-0.791468,-1.134153,-1.516775,-1.88872
4,-0.004301,-0.07033,-0.129073,-0.196883,-0.255237,-0.464528,-0.798815,-1.177218,-1.520773,-1.890441
5,-0.030407,-0.08475,-0.142026,-0.214289,-0.27902,-0.487704,-0.838419,-1.180861,-1.528718,-1.892263
6,-0.011485,-0.085003,-0.146782,-0.200223,-0.277907,-0.523277,-0.911138,-1.242042,-1.564395,-1.903345
7,0.266076,0.213014,0.163175,0.097252,0.07231,-0.389131,-0.996458,-1.340924,-1.636506,-1.929912
8,0.223273,0.187715,0.140869,0.100288,0.050956,-0.124077,-0.711047,-1.308841,-1.643338,-1.935631
9,0.198381,0.15594,0.106158,0.071295,0.032234,-0.122052,-0.427812,-1.004301,-1.614341,-1.939426
10,0.154819,0.121028,0.080049,0.048525,0.01007,-0.151756,-0.431709,-0.746167,-1.291281,-1.888113


,12,13,14,15,16,17,18,19,20,21
A,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay
2,hit,stay,stay,stay,stay,stay,stay,stay,stay,stay
3,hit,stay,stay,stay,stay,stay,stay,stay,stay,stay
4,stay,stay,stay,stay,stay,stay,stay,stay,stay,stay
5,stay,stay,stay,stay,stay,stay,stay,stay,stay,stay
6,stay,stay,stay,stay,stay,stay,stay,stay,stay,stay
7,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay
8,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay
9,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay
10,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay


In [171]:
for starting_usable_ace in [True, False]:
    for starting_action in ["hit", "stay"]:
        print(f"starting_action={starting_action}, starting_usable_ace={starting_usable_ace}")
        display(state_action_values[starting_action][starting_usable_ace])
    display(policy[starting_usable_ace])

starting_action=hit, starting_usable_ace=True


,12,13,14,15,16,17,18,19,20,21
A,-0.335441,-0.359268,-0.390378,-0.406030,-0.421814,-0.445035,-0.402843,-0.333435,-0.288930,-0.168378
2,0.089387,0.061210,0.037535,0.016442,-0.000759,0.008904,0.072697,0.143117,0.193969,0.256565
3,0.112505,0.092119,0.064093,0.035362,0.021500,0.046795,0.107300,0.156422,0.216230,0.262585
4,0.139771,0.130716,0.084480,0.078768,0.057672,0.085445,0.136338,0.184904,0.238592,0.301088
5,0.179229,0.140024,0.129755,0.101685,0.088127,0.118936,0.155208,0.210351,0.273804,0.319352
6,0.186868,0.168403,0.158185,0.134618,0.104062,0.148075,0.202003,0.245877,0.301493,0.343182
7,0.164761,0.136635,0.092068,0.050842,0.008701,0.063085,0.180351,0.232015,0.272805,0.289906
8,0.101427,0.050688,0.008903,-0.044215,-0.062478,-0.071533,0.039004,0.147223,0.204048,0.236327
9,0.009257,-0.039002,-0.072794,-0.112410,-0.154196,-0.146861,-0.099155,0.002783,0.117733,0.169744
10,-0.131475,-0.163244,-0.193504,-0.234482,-0.251480,-0.256185,-0.209086,-0.145351,-0.044017,0.058335


starting_action=stay, starting_usable_ace=True


,12,13,14,15,16,17,18,19,20,21
A,-0.807972,-0.806050,-0.809490,-0.807558,-0.804725,-0.690090,-0.449436,-0.215814,0.030659,0.573236
2,-0.277317,-0.273270,-0.258600,-0.260687,-0.275864,-0.127586,0.146557,0.409066,0.658707,0.884037
3,-0.237353,-0.232902,-0.213679,-0.232256,-0.250974,-0.086963,0.177164,0.410634,0.661894,0.892487
4,-0.190915,-0.205382,-0.204270,-0.190874,-0.211008,-0.063540,0.193353,0.426772,0.667763,0.891272
5,-0.159450,-0.148523,-0.143363,-0.147569,-0.163960,-0.025902,0.220468,0.447716,0.685268,0.897850
6,-0.144476,-0.141340,-0.145488,-0.140487,-0.142409,0.014266,0.289523,0.518213,0.715912,0.905995
7,-0.472380,-0.461858,-0.478956,-0.464663,-0.468913,-0.090555,0.413366,0.621724,0.774602,0.928055
8,-0.510927,-0.512242,-0.511230,-0.504629,-0.511509,-0.369151,0.115748,0.599818,0.793524,0.934379
9,-0.539053,-0.544820,-0.547023,-0.543482,-0.540345,-0.423635,-0.178479,0.289841,0.758158,0.938578
10,-0.581849,-0.584379,-0.567663,-0.586584,-0.575454,-0.466232,-0.229069,-0.012294,0.429598,0.886921


,12,13,14,15,16,17,18,19,20,21
A,hit,hit,hit,hit,hit,hit,hit,stay,stay,stay
2,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
3,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
4,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
5,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
6,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
7,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
8,hit,hit,hit,hit,hit,hit,stay,stay,stay,stay
9,hit,hit,hit,hit,hit,hit,hit,stay,stay,stay
10,hit,hit,hit,hit,hit,hit,hit,stay,stay,stay


starting_action=hit, starting_usable_ace=False


,12,13,14,15,16,17,18,19,20,21
A,-0.556843,-0.583658,-0.614299,-0.644993,-0.665132,-0.695628,-0.729886,-0.797024,-0.879713,-1
2,-0.241740,-0.298204,-0.342997,-0.414967,-0.469716,-0.523631,-0.617802,-0.725014,-0.856839,-1
3,-0.230508,-0.281659,-0.342997,-0.403329,-0.462403,-0.525301,-0.623419,-0.716259,-0.856333,-1
4,-0.202783,-0.263914,-0.321038,-0.389415,-0.455521,-0.531475,-0.612337,-0.727747,-0.850210,-1
5,-0.190741,-0.236339,-0.298877,-0.376765,-0.442364,-0.513308,-0.614341,-0.727949,-0.858560,-1
6,-0.166759,-0.227889,-0.296347,-0.359966,-0.415950,-0.499089,-0.609787,-0.724407,-0.850868,-1
7,-0.208247,-0.266545,-0.313145,-0.362850,-0.410333,-0.484212,-0.585902,-0.721370,-0.852285,-1
8,-0.284139,-0.313499,-0.368618,-0.413045,-0.455875,-0.496812,-0.590810,-0.714691,-0.850716,-1
9,-0.347786,-0.386865,-0.434600,-0.463897,-0.507388,-0.555409,-0.619655,-0.713375,-0.848894,-1
10,-0.417354,-0.456993,-0.493397,-0.537671,-0.564670,-0.613956,-0.675826,-0.756237,-0.855473,-1


starting_action=stay, starting_usable_ace=False


,12,13,14,15,16,17,18,19,20,21
A,-0.810372,-0.805818,-0.812386,-0.811466,-0.801751,-0.688139,-0.455217,-0.211275,0.028946,0.573503
2,-0.266076,-0.265874,-0.275349,-0.282599,-0.271264,-0.134956,0.131566,0.402257,0.652244,0.888113
3,-0.230660,-0.249481,-0.231532,-0.230886,-0.229734,-0.111122,0.168050,0.417894,0.660442,0.888720
4,-0.198482,-0.193584,-0.191965,-0.192531,-0.200283,-0.066947,0.186478,0.449471,0.670563,0.890441
5,-0.160334,-0.151589,-0.156851,-0.162475,-0.163344,-0.025605,0.224078,0.452912,0.670158,0.892263
6,-0.155274,-0.142886,-0.149565,-0.159743,-0.138043,0.024188,0.301351,0.517636,0.713527,0.903345
7,-0.474323,-0.479559,-0.476321,-0.460102,-0.482643,-0.095081,0.410556,0.619554,0.784221,0.929912
8,-0.507412,-0.501214,-0.509487,-0.513333,-0.506831,-0.372736,0.120237,0.594150,0.792622,0.935631
9,-0.546167,-0.542805,-0.540758,-0.535192,-0.539621,-0.433357,-0.191843,0.290927,0.765447,0.939426
10,-0.572173,-0.578021,-0.573445,-0.586196,-0.574739,-0.462200,-0.244117,-0.010070,0.435808,0.888113


,12,13,14,15,16,17,18,19,20,21
A,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay
2,hit,stay,stay,stay,stay,stay,stay,stay,stay,stay
3,hit,stay,stay,stay,stay,stay,stay,stay,stay,stay
4,stay,stay,stay,stay,stay,stay,stay,stay,stay,stay
5,stay,stay,stay,stay,stay,stay,stay,stay,stay,stay
6,stay,stay,stay,stay,stay,stay,stay,stay,stay,stay
7,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay
8,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay
9,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay
10,hit,hit,hit,hit,hit,stay,stay,stay,stay,stay
